In [25]:
import os
import pandas as pd
# imports from other Mosaic modules
from mosaic_sql.crate import EngineInstance, get_engine

In [26]:
os.environ['CRATE_HOST'] = 'ttda.cratedb-dev-cluster.mosaic.hartreepartners.com:4200'
engine = get_engine(EngineInstance.TTDA, timeout=20, debug=False)

In [27]:
date = '2021-08-12'
indicative_settles_sql = f"""
    select * 
     from settles.values_2
    where source='TT'
    and label='Settlement-Indicative'
    and field='Price'
    and date='{date}'
"""

settles_sql = f"""
    select *
    from settles.values_2
    where source='TT'
    and label='Settlement'
    and field='Price'
    and date='{date}'
"""


In [28]:
idf = pd.read_sql(indicative_settles_sql, con=engine)
idf['i_ts'] = pd.to_datetime(idf['epoch_time'], unit='s')
idf['indicative_price'] = idf['value']

idf.drop(columns=['value','epoch_time','label','properties'], inplace=True)
 
sdf = pd.read_sql(settles_sql, con=engine)
sdf['s_ts'] = pd.to_datetime(sdf['epoch_time'], unit='s')
sdf['settle_price'] = sdf['value']
sdf.drop(columns=['value','epoch_time','label','properties'], inplace=True)


In [29]:
merged_df = pd.merge(idf, sdf, how="outer", on=['instrument_key','exchange','date','source','field'], suffixes=("_i","_s"), indicator=True)
# merged_df.drop(columns=['date_s',],inplace=True)

In [30]:
merged_df.to_csv(f'c:\\temp\\is_{date}.csv', index=False)


